# Assessment of Concentrating Solar Thermal and Photovoltaic Technologies for the Optimal Hybridization of Industrial Process Heat Systems

This code is written to model optimal design strategies to reduce fossil fuel consumption in the industrial process heat (IPH) sector by hybridization with solar energy technologies.

In this example, we seek to determine the optimal solar array aperture area and energy storage capacity that maximizes the lifecycle savings associated with augmenting a conventional natural gas IPH system. Energy can be generated using Parabolic Trough Collectors (PTCs) or Photovoltaics (PVs) with resistive heating, and energy can be stored as thermal energy (TES) or electrical energy (EES).  Here, we use user-defined functions, the JuMP modeling language, the EAGO spatial branch-and-bound algorithm with custom upper- and lower-bounding procedures, and the IPOPT algorithm for solving the bounding subproblems.

In this example, we will solve the optimal design problem for PV-EES (fixed, non-tracking) in Firebaugh, CA with the commercial fuel rate, constant IPH demand of $\dot q_p=10^4$ kW, and a minimum solar fraction of 85% ($\xi=0.85$). The code can easily be modified for different technology configurations, process demand, minimum solar fraction, or geographic location.


In [1]:
using JuMP, EAGO, Ipopt;

We will also use the CSV package because our solar resource data (downloaded from the NSRDB) is in a CSV file.

In [2]:
using CSV, DataFrames, Statistics;

The code is organized as follows:  
1. We import the solar resource data for the region we are concerned with.  By default, this is the typical meteorological year (TMY) hourly data (8760 data points) which includes the geographic information (coordinates and timezone) as well as the direct normal irradiance (DNI).  Together, we can model the process of concentrating the solar radiation and collecting it as heat in our system.  
2. Once we have the resource data, we call "PTCmodel.jl" or "pvModel.jl" to simulate the specific thermal power of the solar  technology in units of W/m$^2$ given our geographic region and incident angle data.  This function calls "solarAngles.jl" to calculate the angles of the direct solar radiation incident to the concentrator aperture with respect to each hour in the TMY data. If we are using pvModel, we must specify 0 or 1-axis tracking. If we are using PTCmodel, it is assumed 1-axis tracking because the technology requires it. 
4. We define a function closure for the solar fraction calculated by "iphProcessSmooth.jl", which simulates the performance of the full solar concentrator and thermal storage system hybridized with the industrial process heat system.  The solar fraction is calculated by Eq. 12 in Stuber (2018) and augmented for this example in a forthcoming manuscript (Eq. 14 in the forthcoming paper).
5. We define the custom upper- and lower-bounding optimization subproblems for the spatial branch-and-bound algorithm.  The lower-bounding problem uses the convex capital model for its objective function (Eq. 16 in Stuber (2018)), and the upper-bounding problem uses the nonconvex capital model as its objective function (Eq. 20 in the forthcoming paper).
6. We set up the JuMP model with the EAGO optimizer (with custom bounding procedures) and we solve it.

In [3]:
include("solarAngles.jl")
include("PTCmodel.jl")
include("iphProcessSmooth.jl")
include("lifecycleCost.jl")
include("genModel.jl")
include("smoothMinMaxAbs.jl")
include("locations.jl")

# Step 1: read the data into a table and extract the appropriate data into a vector
place = "Firebaugh" #Firebaugh, Aurora, Weston
track = 0 #0 if no tracking, 1 if tracking, PTC assumes tracking
gen = "PV" #"PV" or "PTC" the method we use for generating energy
stor = "EES" #"EES" or "TES", the energy storage type
qsys = 10 #Average power demand of system in MW
ngFT = 9.87 #9.87 or 19.74 price of natural gas in $/ft3 (commercial) https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SCA_m.htm
xi = 0.85 #minimum solar fraction

input_file, tz, Lat, Long = locations(place)
solData = CSV.File(input_file) |> DataFrame
dniData = convert(Array{Float64,1},solData[:,7]) #DNI
dhiData = convert(Array{Float64,1},solData[:,6]) #DHI
Temp = convert(Array{Float64,1}, solData[:, 11]) #Temp


# Step 2: get the specific thermal power potential for the region [W/m^2]
q, GHI = genModel(dniData, dhiData, Temp, tz, Lat, Long, track, gen)

# Step 3: define the solar fraction closure
SolarFrac(xts,xa) = iphProcessSmooth(q,xts,xa, qsys,stor);

"photovoltaic Power/GHI"

42.20522371847136

245.63667954571486

Step 4 is a bit more complicated as we need to define the custom bounding procedures for the branch-and-bound algorithm.  First, we define an extension type called $\texttt{SolarExt}$ which allows EAGO to dispatch to the custom bounding routines we plan on defining.  

In [4]:
import EAGO: Optimizer, GlobalOptimizer
struct SolarExt <: EAGO.ExtensionType end

Now, we define the custom lower-bounding problem with the convex capital model.

In [5]:
# Lower Problem Definition
import EAGO.lower_problem!
function lower_problem!(t::SolarExt,opt::EAGO.GlobalOptimizer)
    
    # Get active node
    n = opt._current_node
    
    # Creates model, adds variables, and register nonlinear expressions
    m = JuMP.Model(JuMP.optimizer_with_attributes(Ipopt.Optimizer,
                    "tol"=>1.0e-4,
                    "print_level"=>0))
    xL = n.lower_variable_bounds; xU = n.upper_variable_bounds
    @variable(m, xL[i] <= x[i=1:2] <= xU[i])
    # define convex lifecycle savings objective function closure
    
    flcsC(xts,xa) = lifecycleCost(q, xL, xU, xts, xa, :convex, track, gen, stor, ngFT, qsys)
    
    JuMP.register(m, :flcsC, 2, flcsC, autodiff=true) #register: be aware of exteneral functions
    JuMP.register(m, :SolarFrac, 2, SolarFrac, autodiff=true)

    # Define nonlinear function
    
    @NLobjective(m, Max, flcsC(x[1], x[2]))
    @NLconstraint(m, g1, SolarFrac(x[1], x[2]) >= xi)# declare constraints
    JuMP.optimize!(m)

    # Get primal status, termination status, determine if a global solution was obtained
    tstatus = MOI.get(m, MOI.TerminationStatus())
    pstatus = MOI.get(m, MOI.PrimalStatus())
    solution = JuMP.value.(x)
    
    if EAGO.local_problem_status(tstatus, pstatus) == EAGO.LRS_FEASIBLE
        opt._lower_objective_value = -1.0*(JuMP.objective_value(m)-1e-4) # Multiplied by -1 because EAGO expects "Min"
        opt._lower_solution[1:length(solution)] = solution
        opt._lower_feasibility = true
        opt._cut_add_flag = false
    else
        opt._lower_feasibility = false
        opt._lower_objective_value = Inf
        opt._cut_add_flag = false
    end
    return
end;

Now, we define the upper-bounding problem with the nonconvex capital pricing model.

In [6]:
# Upper Problem Definition
import EAGO.upper_problem!
function upper_problem!(t::SolarExt,opt::EAGO.GlobalOptimizer)
    
    # Get active node
    n = opt._current_node
    
    # Creates model, adds variables, and register nonlinear expressions
    m = JuMP.Model(JuMP.optimizer_with_attributes(Ipopt.Optimizer,
                    "tol"=>1.0e-4,
                    "print_level"=>0))
    xL = n.lower_variable_bounds; xU = n.upper_variable_bounds
    @variable(m, xL[i] <= x[i=1:2] <= xU[i])
    # define lifecycle savings objective function cover

    flcs(xts,xa) = lifecycleCost(q, xL, xU, xts, xa, :nonconvex, track, gen, stor, ngFT, qsys)
    
    JuMP.register(m, :flcs, 2, flcs, autodiff=true)
    JuMP.register(m, :SolarFrac, 2, SolarFrac, autodiff=true)

    # Define nonlinear function
    @NLobjective(m, Max, flcs(x[1], x[2]))
    @NLconstraint(m, g1, SolarFrac(x[1], x[2]) >= xi)# declare constraints
    JuMP.optimize!(m)

    # Get primal status, termination status, determine if a global solution was obtained
    tstatus = MOI.get(m, MOI.TerminationStatus())
    pstatus = MOI.get(m, MOI.PrimalStatus())
    solution = JuMP.value.(x)

    if EAGO.local_problem_status(tstatus, pstatus) == EAGO.LRS_FEASIBLE
        opt._upper_objective_value = -1.0*JuMP.objective_value(m) # Multiplied by -1 because EAGO expects "Min"
        opt._upper_solution[1:length(solution)] = solution
        opt._upper_feasibility = true
    else
        opt._upper_feasibility = false
        opt._upper_objective_value = Inf
        opt._cut_add_flag = false
    end
    return
end;

Since we have defined custom bounding routines, we'll disable some unnecessary EAGO subroutines.

In [7]:
import EAGO: preprocess!, postprocess!, cut_condition
function EAGO.preprocess!(t::SolarExt, x::GlobalOptimizer)
    x._preprocess_feasibility = true
    return
end
function EAGO.postprocess!(t::SolarExt, x::GlobalOptimizer)
    x._postprocess_feasibility = true
    return
end
EAGO.cut_condition(t::SolarExt, x::GlobalOptimizer) = false

Now, we define the JuMP model and the variables (and bounds).  We must specify that our new custom extension $\texttt{SolarExt}$ to EAGO's default routines should be used and that we will be branching on both of our decision variables.  The latter is required for custom routines since no expressions will be provided to the EAGO optimizer and therefore it cannot infer which variables should be branched on. 

In [8]:
factory = () -> EAGO.Optimizer(SubSolvers(;t=SolarExt()))

m = JuMP.Model(optimizer_with_attributes(factory,
                    "relative_tolerance"=>1e-2,
                    "verbosity"=>1,
                    "output_iterations"=>1,
                    "branch_variable"=>Bool[true; true],
)) 
x_L = [0.0001, 0.0001] #add 5 zeros to test SF > 0.85 constraint
x_U = [100, 500000.0] #add 5 zeros to test SF > 0.85 constraint
@variable(m, x_L[i] <= x[i=1:2] <= x_U[i]);
JuMP.register(m, :SolarFrac, 2, SolarFrac, autodiff=true)
@NLconstraint(m, g1, SolarFrac(x[1], x[2]) >= xi);

Now, let's solve the problem and print the results.

In [9]:
@time JuMP.optimize!(m)

println("xts* = ", JuMP.value(x[1]), " xa* = ",
         JuMP.value(x[2])," f* = ",-1.0*JuMP.objective_value(m)," SF* = ",
         SolarFrac(JuMP.value(x[1]),JuMP.value(x[2])))
TermStatus = JuMP.termination_status(m)
PrimStatus = JuMP.primal_status(m)
println("Algorithm terminated with a status of $TermStatus and a result code of $PrimStatus")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

-----------------------------------------------------------------------------------------------------------------------------
|  Iteration #  |     Nodes    | Lower Bound  |  Upper Bound  |      Gap     |     Ratio    |     Time     |    Time Left   |
-----------------------------------------------------------------------------------------------------------------------------
|            1  |            2 |    4.896E+07 |          Inf |         Inf |          Inf |    1.596E+01 |    3.584E+03 |


|            2  |            3 |    4.896E+07 |    5.502E+07 |   6.056E+06 |    1.101E-01 |    2.287E+01 |    3.577E+03 |


|            3  |            2 |    5.354E+07 |    5.502E+07 |   1.471E+06 |    2.674E-02 |    2.295E+01 |    3.577E+03 |
|            4  |            3 |    5.354E+07 |    5.502E+07 |   1.471E+06 |    2.674E-02 |    2.321E+01 |    3.577E+03 |


|            5  |            4 |    5.420E+07 |    5.502E+07 |   8.198E+05 |    1.490E-02 |    2.336E+01 |    3.577E+03 |


|            6  |            3 |    5.420E+07 |    5.502E+07 |   8.198E+05 |    1.490E-02 |    3.306E+01 |    3.567E+03 |
|            7  |            4 |    5.420E+07 |    5.502E+07 |   8.145E+05 |    1.480E-02 |    4.274E+01 |    3.557E+03 |


|            8  |            3 |    5.420E+07 |    5.502E+07 |   8.145E+05 |    1.480E-02 |    4.283E+01 |    3.557E+03 |
|            9  |            4 |    5.428E+07 |    5.502E+07 |   7.395E+05 |    1.344E-02 |    5.250E+01 |    3.548E+03 |


|           10  |            3 |    5.428E+07 |    5.502E+07 |   7.395E+05 |    1.344E-02 |    5.260E+01 |    3.547E+03 |
|           11  |            4 |    5.428E+07 |    5.502E+07 |   7.395E+05 |    1.344E-02 |    5.431E+01 |    3.546E+03 |


|           12  |            3 |    5.428E+07 |    5.502E+07 |   7.395E+05 |    1.344E-02 |    5.439E+01 |    3.546E+03 |
|           13  |            4 |    5.491E+07 |    5.502E+07 |   1.056E+05 |    1.919E-03 |    5.521E+01 |    3.545E+03 |
 
Relative Tolerance Achieved
First Solution Found at Node 5
LBD = 5.4909619232100055e7
UBD = 5.501517150574002e7
Solution is:
    X[1] = 14.86766956690542
    X[2] = 267031.33546399354
 
 61.063245 seconds (34.42 M allocations: 105.782 GiB, 7.01% gc time, 16.97% compilation time: 5% of which was recompilation)


xts* = 14.86766956690542 xa* = 267031.33546399354 f* = -5.501517150574002e7 SF* = 

0.8499999900000049
Algorithm terminated with a status of OPTIMAL and a result code of FEASIBLE_POINT
